In [37]:
import time
import pandas as pd
import numpy as np
from numpy import array
import torch
import torch.nn as nn
from sklearn.model_selection import KFold

In [38]:
save_path = "/Users/Brody1/Documents/Northwestern/Jiping/benchmarks/deep-learning/"
model_name = "fourier_one_features_2"
kf = KFold(n_splits = 10, shuffle =True)
num_epochs = 60

In [39]:
#### define functions ####

# # Fully connected neural network with one hidden layer
# class NeuralNet(nn.Module):
#     def __init__(self, input_size, hidden_size, out_dim):
#         super(NeuralNet, self).__init__()
#         self.fc1 = nn.Linear(input_size, hidden_size)
#         self.relu = nn.ReLU()
#         self.fc2 = nn.Linear(hidden_size, out_dim)

#     def forward(self, x):
#         out = self.fc1(x)
#         out = self.relu(out)
#         out = self.fc2(out)
#         return out


# Fully connected neural network with one hidden layer
class NeuralNet(nn.Module):
    def __init__(self, input_size_one_hot, input_size_extra, hidden_size_one_hot, hidden_size_extra, out_dim):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size_extra, hidden_size_extra)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size_extra, out_dim)

        self.conv1 = nn.Conv2d(input_size_one_hot, out_channels=hidden_size_one_hot, kernel_size=(3,4))



    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out


In [40]:
def display_fits(fits):
    print(f"Average correlation on tiling: {np.mean([fits[0][i] for i in range(fits[0].size) if (i % 4) == 2])}",
          f"\nAverage MSE on tiling: {np.mean([fits[1][i] for i in range(fits[1].size) if (i % 4) == 2])}",
          f"\nAverage correlation on random: {np.mean([fits[0][i] for i in range(fits[0].size) if (i % 4) == 1])}",
          f"\nAverage MSE on random: {np.mean([fits[1][i] for i in range(fits[1].size) if (i % 4) == 1])}",
          f"\nAverage correlation on ChrV: {np.mean([fits[0][i] for i in range(fits[0].size) if (i % 4) == 3])}",
          f"\nAverage MSE on ChrV: {np.mean([fits[1][i] for i in range(fits[1].size) if (i % 4) == 3])}",
          f"\nAverage correlation on CN: {np.mean([fits[0][i] for i in range(fits[0].size) if (i % 4) == 0])}",
          f"\nAverage MSE on CN: {np.mean([fits[1][i] for i in range(fits[1].size) if (i % 4) == 0])}")

In [41]:
# Construct sine and cosine matrices for Fourier features:
cos_matrix, sin_matrix = [[] for x in range(50)], [[] for x in range(50)]
for n in range(50):
    for k in range(50):
        cos_matrix[n].append(np.cos(2*np.pi*(n+1)*k/50))
        sin_matrix[n].append(np.sin(2*np.pi*(n+1)*k/50))

In [95]:
def dnaOneFourier(sequence):
    seq_array = array(list(sequence))
    code = {"A": [1], "C": [0], "G": [0], "T": [1],
            "a": [1], "c": [0], "g": [0], "t": [1]}
    AT_encoded_seq = []
    for char in seq_array:
        AT_encoded_seq.append(code[char][0])
    one_fourier_cos = np.matmul(array(AT_encoded_seq), array(cos_matrix).transpose())
    one_fourier_sin = np.matmul(array(AT_encoded_seq), array(sin_matrix).transpose())
    return array((one_fourier_cos[5], one_fourier_sin[5]), dtype = float)

In [96]:
def dnaOneHot(sequence):
    seq_array = array(list(sequence))
    code = {"A": [0], "C": [1], "G": [2], "T": [3], "N": [4],
            "a": [0], "c": [1], "g": [2], "t": [3], "n": [4]}
    onehot_encoded_seq = []
    for char in seq_array:
        onehot_encoded = np.zeros(5)
        onehot_encoded[code[char]] = 1
        onehot_encoded_seq.append(onehot_encoded[0:4])
    return onehot_encoded_seq

In [97]:
def find_c0new(dat):
  mat = np.empty((3,3), float)
  k = 2*np.pi/10.4
  n = array([26, 29, 31])
  mat[0:3,0] = 1
  mat[0:3, 1] = np.sin(n*k)
  mat[0:3, 2] = np.cos(n*k)
  inv_mat = np.linalg.inv(mat)
  c0A1A2 = array(np.matmul(dat[["n=26", "n=29", "n=31"]], inv_mat))
  c0Aphi = c0A1A2
  c0Aphi[:,0] = c0A1A2[:,0]
  c0Aphi[:,1] = np.sqrt(c0A1A2[:,1]**2 + c0A1A2[:,2]**2)
  c0Aphi[:,2] <- np.sign(c0A1A2[:,2]) * np.arccos(c0A1A2[:,1]/c0Aphi[:,1])
  return c0Aphi[:,0]

In [118]:
#### preparing data ####

data_cerevisiae_nucle = pd.read_csv("/Users/Brody1/Documents/Northwestern/Jiping/cycle1.txt",delimiter = ",")
X1 = []
for sequence_nt in data_cerevisiae_nucle["Sequence"]:
    X1.append([dnaOneFourier(sequence_nt), array(dnaOneHot(sequence_nt)).reshape(50, 4, 1)])
# X1 = np.float32(array(X1))
# X1 = X1.reshape((X1.shape[0],50,4,1))
### COMPUTE EIGENVECTORS:

# X1_reverse = np.flip(X1,[1,2])
# Y1 = data_cerevisiae_nucle["C0"].values.astype(float)
Y1 = np.float32(find_c0new(data_cerevisiae_nucle).astype(float))


In [119]:
print(array(X1).shape)
print(array(X1)[0, 1].shape)

(19907, 2)
(50, 4, 1)


/Users/Brody1/anaconda3/envs/myenv/lib/python3.7/site-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  """Entry point for launching an IPython kernel.
/Users/Brody1/anaconda3/envs/myenv/lib/python3.7/site-packages/ipykernel_launcher.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  


In [ ]:
data_random_library = pd.read_csv("/Users/Brody1/Documents/Northwestern/Jiping/cycle3.txt",delimiter = ",")
X3 = []
for sequence_nt in data_random_library["Sequence"]:
    X3.append(dnaOneFourier(sequence_nt))
    X3.append(dnaOneHot(sequence_nt))
X3 = np.float32(array(X3))
# X3 = X3.reshape((X3.shape[0],50,2,1))
# X3_reverse = np.flip(X3,[1,2])
# Y3 = data_random_library["C0"].values.astype(float)
Y3 = np.float32(find_c0new(data_random_library).astype(float))

In [60]:
data_tiling = pd.read_csv("/Users/Brody1/Documents/Northwestern/Jiping/cycle5.txt",delimiter = ",")
X5 = []
for sequence_nt in data_tiling["Sequence"]:
    X5.append(dnaOneFourier(sequence_nt))
    X5.append(dnaOneHot(sequence_nt))
X5 = np.float32(array(X5))
# X5 = X5.reshape((X5.shape[0],50,4,1))
# X5_reverse = np.flip(X5,[1,2])
# Y5 = data_tiling["C0"].values.astype(float)
Y5 = np.float32(find_c0new(data_tiling).astype(float))

/Users/Brody1/anaconda3/envs/myenv/lib/python3.7/site-packages/ipykernel_launcher.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  


ValueError: setting an array element with a sequence.

In [ ]:
data_chr5 = pd.read_csv("/Users/Brody1/Documents/Northwestern/Jiping/cycle6.txt",delimiter = ",")
X6 = []
for sequence_nt in data_chr5["Sequence"]:
    X6.append(dnaOneFourier(sequence_nt))
    X6.append(dnaOneHot(sequence_nt))
X6 = np.float32(array(X6))
# X6 = X6.reshape((X6.shape[0],50,4,1))
# X6_reverse = np.flip(X6,[1,2])
# Y6 = data_chr5["C0"].values.astype(float)
Y6 = np.float32(find_c0new(data_chr5).astype(float))

In [61]:
#### tiling

VALIDATION_LOSS = []
fold_var = 1
n = Y5.shape[0]
hidden_size = 50
learning_rate = 0.01

fits = []
detrend = []
times = []

for train_index, val_index in kf.split(Y5):
    training_X = np.float32(X5[train_index])
    validation_X = np.float32(X5[val_index])
    training_Y = np.float32(Y5[train_index])
    validation_Y = np.float32(Y5[val_index])

    # CREATE NEW MODEL
    model = NeuralNet(input_size=202, hidden_size=hidden_size, out_dim=1)
    # Loss and optimizer
    criterion = nn.MSELoss()
    optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)
    # CREATE CALLBACKS

    for epoch in range(num_epochs):
        inputs = torch.from_numpy(training_X)
        inputs = inputs.reshape(inputs.shape[0], 202)
        targets = torch.from_numpy(training_Y)
        targets = targets.reshape(targets.shape[0], 1)

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (epoch + 1) % 5 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch + 1, num_epochs, loss.item()))
    
    pred_Y = model(torch.from_numpy(training_X)).detach().numpy()
    pred_Y = pred_Y.reshape(pred_Y.shape[0])

    start_time = time.process_time()
    fit = model(torch.from_numpy(X1)).detach().numpy()
    fit = fit.reshape(fit.shape[0])
    fit_tmp =[np.corrcoef(fit, Y1)[0,1],np.mean(np.square(fit-Y1)),np.mean(fit),np.std(fit)]
    fits.append(fit_tmp)
    time0 = time.process_time() - start_time
    times.append([time0])
    
    start_time = time.process_time()
    fit = model(torch.from_numpy(X3)).detach().numpy()
    fit = fit.reshape(fit.shape[0])
    fit_tmp =[np.corrcoef(fit, Y3)[0,1],np.mean(np.square(fit-Y3)),np.mean(fit),np.std(fit)]
    fits.append(fit_tmp)
    time0 = time.process_time() - start_time
    times.append([time0])
    
    start_time = time.process_time()
    fit = model(torch.from_numpy(validation_X)).detach().numpy()
    fit = fit.reshape(fit.shape[0])
    fit_tmp =[np.corrcoef(fit, validation_Y)[0,1],np.mean(np.square(fit-validation_Y)),np.mean(fit),np.std(fit)]
    fits.append(fit_tmp)
    time0 = time.process_time() - start_time
    times.append([time0])
    
    start_time = time.process_time()
    fit = model(torch.from_numpy(X6)).detach().numpy()
    fit = fit.reshape(fit.shape[0])
    fit_tmp =[np.corrcoef(fit, Y6)[0,1],np.mean(np.square(fit-Y6)),np.mean(fit),np.std(fit)]
    fits.append(fit_tmp)
    time0 = time.process_time() - start_time
    times.append([time0])
    
    fold_var += 1

fits = array(fits)
fits = pd.DataFrame((fits))
fits.to_csv(save_path +model_name+"_fits_tiling.txt", index = False)

with open(save_path +model_name+"_time_tiling.txt", "w") as file:
    for row in times:
        s = " ".join(map(str, row))
        file.write(s+'\n')


TypeError: only integer scalar arrays can be converted to a scalar index

In [57]:
display_fits(fits)

Average correlation on tiling: 0.009142444649559739 
Average MSE on tiling: 0.16104955077171326 
Average correlation on random: 0.007457088024553468 
Average MSE on random: 0.12480362206697464 
Average correlation on ChrV: 0.006018485089171445 
Average MSE on ChrV: 0.2580263018608093 
Average correlation on CN: 0.007645504574162086 
Average MSE on CN: 0.2234574779868126
